In [1]:
#| tags: [parameters]
root = r"inputs"

In [2]:
#| echo: false
import pandas as pd
import os
import numpy as np
import Form as form
import FormComparator as comp

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\langhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\langhe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# WHO verbal autopsy comparisons

In [3]:
in_xlsx1 = os.path.join(root, r"WHOVA2016_v1_5_3_XLS_form_for_ODK.xlsx")
if os.path.exists(in_xlsx1) and in_xlsx1.endswith('.xlsx'):
    f1 = form.Form(in_xlsx1,
                   survey_type = "WHO VA autopsy")

Create Form object from WHOVA2016_v1_5_3_XLS_form_for_ODK.xlsx
	 - Info: survey sheet with 14 columns and 600 rows
	 - Info: choices sheet with 4 columns and 356 rows
	 - Info: settings sheet with 5 columns
	 - Info: no entities sheet found
Create Form object from WHOVA2022_XLS_form_for_ODK.xlsx
	 - Info: survey sheet with 18 columns and 509 rows
	 - Info: choices sheet with 4 columns and 332 rows
	 - Info: settings sheet with 5 columns
	 - Info: no entities sheet found


In [ ]:
in_xlsx2 = os.path.join(root, r"WHOVA2022_XLS_form_for_ODK.xlsx")
if os.path.exists(in_xlsx2) and in_xlsx2.endswith('.xlsx'):
    f2 = form.Form(in_xlsx2,
                   survey_type = "WHO VA autopsy")

## Generic comparison

In [4]:
comparison = comp.FormComparator(cur_form = f2,
                                 ref_form = f1,
                                 output_dir = "outputs")

## Comparison based on variable names

Note:

* Identified added questions are displayed with the variable name and label of the closest label match in the reference XLSForm.
* Identified removed questions are displayed with the variable name and label of the closest label match in the target XLSForm.

In [5]:
#| tbl-cap: "SPA exit interview - Senegal vs. Tanzania - Added variable names"
f1.detectAddedQuestions(f2)

,row,name,label,index,name__skrub_1191d02f__,label::English,skrub_Joiner_distance,skrub_Joiner_rescaled_distance,skrub_Joiner_match_accepted
0,16,Id10007b,age va respondent,7,Id10007a,[ sex va respondent ],0.711848,0.521547,True
1,138,Id10077_a,how long injury accident s/he die,127,Id10098,injury accidental,0.812582,0.595351,True
2,139,Id10077_b,[ interviewer click `` ok '' confirm answer : ...,14,Id10012,,1.000000,0.732666,True
3,239,Id10189_1,s/he vomit every time s/he ate and/or drank,14,Id10012,,1.000000,0.732666,True
4,333,Id10262_c,swallowing become impossible,14,Id10012,,1.000000,0.732666,True
5,384,Id10304_a,faint when sharp abdominal pain,14,Id10012,,1.000000,0.732666,True
6,389,Id10322_a,foul smelling vaginal discharge pregnancy,421,Id10322,foul smelling vaginal discharge pregnancy deli...,0.335637,0.245910,True
7,397,Id10322_b,foul smelling vaginal discharge delivery/abortion,421,Id10322,foul smelling vaginal discharge pregnancy deli...,0.685717,0.502402,True
8,395,Id10329_a,excessive bleeding delivery,427,Id10328,excessive bleeding labour delivery,0.433849,0.317867,True
9,396,Id10329_b,excessive bleeding abortion miscarriage,428,Id10329,excessive bleeding delivery abortion,0.744946,0.545797,True


In [6]:
#| tbl-cap: "SPA exit interview - Senegal vs. Tanzania - Removed variable names"
f1.detectDeletedQuestions(f2)

,row,name,type,label,group_id_x,group_lbl_x
0,72,Id10060,date,date marriage,0,
1,71,Id10060_check,select_one YES_NO,[ date marriage available ],0,
2,81,Id10069,select_one YES_NO,[ need collect civil registration numbers dece...,0,
3,109,Id10080,select_one select_292,her/his role road traffic accident,0,
4,110,Id10081,select_one select_293,counterpart hit road traffic accident,0,
...,...,...,...,...,...,...
103,572,Id10459,select_one YES_NO_DK_REF,"course illness , total costs care treatment pr...",0,
104,62,Id10488,select_one YES_NO_DK_REF,"two weeks death , ( ) travel area where covid-...",0,
105,484,babmove,begin group,baby moving,0,
106,562,back_context,begin group,background context,0,


In [7]:
major, minor = f1.detectModifiedLabels(f2)

In [8]:
#| tbl-cap: "SPA exit interview - Senegal vs. Tanzania - Variable names with major label modifications"
major

,name,row1,label1,row2,label2
0,Id10007,6,[ name va respondent ],14,full name va respondent
1,Id10012,14,,19,date interview
2,Id10061,73,name father,79,full name father
3,Id10062,74,name mother,80,full name mother
4,Id10109,96,baby ever move,99,baby ever move delivered
...,...,...,...,...,...
98,Id10465,578,[ duration ( ia ) : ],495,[ duration immediate cause death ( ia ) : ]
99,Id10467,580,[ duration ( ib ) : ],497,[ duration first antecedent cause death ( ib )...
100,Id10469,582,[ duration ( ic ) : ],499,[ duration second antecedent cause death ( ic ...
101,Id10471,584,[ duration ( id ) : ],501,[ duration third antecedent cause death ( id )...


In [9]:
#| tbl-cap: "SPA exit interview - Senegal vs. Tanzania - Variable names with minor label modifications"
minor

,name,row1,label1,row2,label2
0,Id10116,103,"baby ’ body soft , pulpy discoloured skin peel...",106,"babys body soft , discoloured skin peeling away"
1,Id10201_unit,258,how long death ( ) usually protruding belly ( ...,253,how long death ( ) usually protruding abdomen
2,Id10201,261,calculated number days protruding belly ( abdo...,256,calculated number days protruding abdomen
3,Id10203,262,how rapidly ( ) develop protruding belly ( abd...,257,how rapidly ( ) develop protruding abdomen
4,Id10209_a,271,[ enter how long death ( ) stiff neck days ] :,266,[ enter how long death ( ) stiff painful neck ...
5,Id10209_b,272,[ enter how long death ( ) stiff neck months ] :,267,[ enter how long death ( ) stiff painful neck ...
6,Id10209,273,how many days death ( ) stiff neck,268,how many days death ( ) stiff painful neck
7,Id10232_units,305,how long ulcer foot ooze pus,289,how long ulcer foot pus
8,Id10232_a,306,[ enter how long ulcer foot oozed pus days ] :,290,[ enter how long ulcer foot pus days ] :
9,Id10232_b,307,[ enter how long ulcer foot oozed pus months ] :,291,[ enter how long ulcer foot pus months ] :


In [10]:
#| tbl-cap: "SPA exit interview - Senegal vs. Tanzania - Variable names with type modifications"
f1.detectModifiedTypes(f2)

,name,row1,type1,row2,type2
0,age_child_unit,45,select_one units,51,select_one unitsc
1,Id10178_unit,221,select_one M_H_D_DK,218,select_one H_D_DK_REF
2,Id10199,256,select_one select_100,251,select_multiple select_100
3,Id10260,343,select_multiple select_161,327,select_one YES_NO_DK_REF
4,Id10262_units,347,select_one units_2,329,select_one units_5
5,Id10310,407,select_one YES_NO_DK_REF2,382,select_one confirm
6,Id10317,416,select_one YES_NO_DK_REF,387,select_one select_533
7,Id10376,482,select_one YES_NO_DK_REF,98,select_one select_532
8,Id10414,520,select_one select_306,461,select_one YES_NO_DK_REF


# RCT Day 0 form comparison

In [11]:
in_xlsx3 = os.path.join(root, r"06 Tanzania\1-RCT-LS\01a-TIMCI-CRF-Facility-ref-form\01a-TIMCI-CRF-Facility-ref-form.xlsx")
in_xlsx4 = os.path.join(root, r"05 Senegal\1-RCT-LS\01a-TIMCI-CRF-Facility-Senegal-form\01a-TIMCI-CRF-Facility-Senegal-form.xlsx")
f3 = form.Form(in_xlsx3,
               survey_type = "RCT/LS day 0 form")

f4 = form.Form(in_xlsx4,
               survey_type = "RCT/LS day 0 form")

Create Form object from 01a-TIMCI-CRF-Facility-ref-form.xlsx
	 - Info: no survey sheet found
	 - Info: no choices sheet found
	 - Info: no settings sheet found
	 - Info: no entities sheet found


AttributeError: 'NoneType' object has no attribute 'at'

In [ ]:
## Generic comparison

In [ ]:
print(f3.compare(f4))

## Comparison based on variable names

In [ ]:
#| tbl-cap: "Main Day 0 - Senegal vs. Tanzania - Added variable names"
f3.detectAddedQuestions(f4)

In [ ]:
#| tbl-cap: "Main Day 0 - Senegal vs. Tanzania - Removed variable names"
f3.detectDeletedQuestions(f4)

In [ ]:
major, minor = f3.detectModifiedLabels(f4)

In [ ]:
#| tbl-cap: "Main Day 0 - Senegal vs. Tanzania - Variable names with major label modifications"
major

In [ ]:
#| tbl-cap: "Main Day 0 - Senegal vs. Tanzania - Variable names with minor label modifications"
minor

In [ ]:
#| tbl-cap: "Main Day 0 - Senegal vs. Tanzania - Variable names with type modifications"
f3.detectModifiedTypes(f4)

# Inter-form tests

In [ ]:
#| tbl-cap: "Tanzania - Main Day 0 vs. SPA exit interview - Added variable names"
f3.detectAddedQuestions(f1)

In [ ]:
#| tbl-cap: "Tanzania - Main Day 0 vs. SPA exit interview - Removed variable names"
f3.detectDeletedQuestions(f1)

In [ ]:
f3.detectSimilarLabels(f1)

# Test for choices

In [ ]:
#choices_df["list_name"].dropna().unique()